In [125]:
import pandas as pd

In [126]:
df = pd.read_csv('Cyber1_train.csv')

In [127]:
df.head()

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,151143,0.000003,udp,-,INT,2,0,104,0,333333.321500,...,1,2,0,0,0,1,3,0,Fuzzers,1
1,80153,2.308886,tcp,-,FIN,10,8,564,354,7.362858,...,1,1,0,0,0,1,1,0,Reconnaissance,1
2,164542,0.000003,udp,dns,INT,2,0,114,0,333333.321500,...,17,17,0,0,0,18,17,0,Generic,1
3,32390,0.004722,tcp,-,FIN,18,18,3960,2456,7412.113355,...,1,4,0,0,0,7,7,0,Normal,0
4,161063,0.000002,udp,dns,INT,2,0,114,0,500000.001300,...,16,29,0,0,0,16,29,0,Generic,1


In [128]:
df['attack_cat'].value_counts()

attack_cat
Normal            35840
Generic           25585
Exploits          21514
Fuzzers           11593
DoS                7837
Reconnaissance     6655
Analysis           1257
Backdoor           1133
Shellcode           722
Worms                81
Name: count, dtype: int64

In [129]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

In [130]:
df['attack_cat'] = le.fit_transform(df['attack_cat'])

In [131]:
df['proto'] = le.fit_transform(df['proto'])
df['service'] = le.fit_transform(df['service'])
df['state'] = le.fit_transform(df['state'])

In [132]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_scaled = scaler.fit_transform(df)

In [133]:
df_scaled

array([[ 1.25552432, -0.20911086,  0.42020499, ..., -0.12655666,
        -0.42596238,  0.68501913],
       [-0.14726096,  0.14997587,  0.15173381, ..., -0.12655666,
         1.4883947 ,  0.68501913],
       [ 1.52029288, -0.20911086,  0.42020499, ..., -0.12655666,
         0.21215665,  0.68501913],
       ...,
       [-0.95632759, -0.20764551,  0.15173381, ..., -0.12655666,
         0.85027567, -1.45981324],
       [ 1.63703686, -0.20911055,  0.42020499, ..., -0.12655666,
         0.21215665,  0.68501913],
       [-0.1796086 , -0.03344323,  0.15173381, ..., -0.12655666,
        -0.42596238,  0.68501913]])

In [134]:
y=df['label']

In [135]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 5)

In [136]:
df_pca = pca.fit_transform(df_scaled)

In [137]:
df_pca = pd.DataFrame(data=df_pca, columns=['PC1', 'PC2', 'PC3', 'PC4', 'PC5'])

In [138]:
df_pca.head()

,PC1,PC2,PC3,PC4,PC5
0,-1.458258,0.227929,-2.002504,1.390345,0.655377
1,3.260958,-2.547374,-0.186730,-0.360664,0.448470
2,-4.460075,-0.572127,0.819549,-0.143380,0.058783
3,1.658383,2.348215,0.337869,-1.422528,-1.045005
4,-5.340501,-0.689697,1.269179,-0.312700,0.013950


In [139]:
y

0         1
1         1
2         1
3         0
4         1
         ..
112212    0
112213    1
112214    0
112215    1
112216    1
Name: label, Length: 112217, dtype: int64

In [140]:
df_pca.shape

(112217, 5)

In [141]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_pca, y, test_size = 0.2, random_state = 32)

In [142]:
from sklearn.svm import SVC

model = SVC()

In [143]:
model.fit(X_train,y_train)
model.score(X_test, y_test)

0.9816432008554625

In [144]:
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_test)

In [145]:
print(accuracy_score(y_test, y_pred))

0.9816432008554625


In [146]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.97      0.97      7185
           1       0.98      0.99      0.99     15259

    accuracy                           0.98     22444
   macro avg       0.98      0.98      0.98     22444
weighted avg       0.98      0.98      0.98     22444

